In [1]:
from datasets import load_dataset

dataset  = load_dataset("csv",data_files="./Datasets/dataset_set1/job_postings.csv")

d:\MultiLingualMOOC\FineTuning\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['job_link', 'last_processed_time', 'last_status', 'got_summary', 'got_ner', 'is_being_worked', 'job_title', 'company', 'job_location', 'first_seen', 'search_city', 'search_country', 'search_position', 'job_level', 'job_type'],
        num_rows: 12217
    })
})

In [3]:
from transformers import MT5Tokenizer

In [4]:
tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
d:\MultiLingualMOOC\FineTuning\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
import torch

In [6]:
print(torch.__version__)

2.4.1+cu121


In [7]:
def tokenize(data):
    input_text = [
        f"Job Title: {job_title} | "
        f"Company: {company} | "
        f"Location: {job_location} | "
        f"First Seen: {first_seen} | "
        f"Search City: {search_city} | "
        f"Search Country: {search_country} | "
        f"Search Position: {search_position} | "
        f"Job Level: {job_level} | "
        f"Job Type: {job_type}"
        for job_title, company, job_location, first_seen, search_city, search_country, search_position, job_level, job_type
        in zip(data['job_title'], data['company'], data['job_location'], data['first_seen'], data['search_city'], data['search_country'], data['search_position'], data['job_level'], data['job_type'])
    ]
    
    # Tokenize input text
    input_feature = tokenizer(input_text, truncation=True, max_length=512, padding="max_length")
    
    # Tokenize labels
    labels = tokenizer(data['job_title'], truncation=True, max_length=512, padding="max_length")
    
    # Return the tokenized inputs and labels
    return {
        "input_ids": input_feature['input_ids'],
        "attention_mask": input_feature['attention_mask'],
        "labels": labels['input_ids']
    }

In [8]:
tokenized_dataset = dataset.map(tokenize,batched=True)

In [9]:
from transformers import (
    MT5ForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    MT5Config
)

In [10]:
device  = torch.device("cuda")

In [11]:
config = MT5Config.from_pretrained(
    "google/mt5-small",
    max_length = 512,
    length_penalty = 0.6,
    no_repeat_ngram_size = 2,
    num_beams = 15
)

In [12]:
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small",config=config).to(device)

In [13]:
for param in model.parameters():
    param.data = param.data.contiguous()

In [14]:
from transformers import DataCollatorForSeq2Seq

In [15]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    return_tensors='pt',
    padding=True
)

In [16]:
training_args = Seq2SeqTrainingArguments(
  output_dir = "jobposting",
  log_level = "error",
  num_train_epochs = 7,
  learning_rate = 5e-5,
  lr_scheduler_type = "linear",
  warmup_steps = 90,
  optim = "adafactor",
  weight_decay = 0.01,
  per_device_train_batch_size = 2,
  per_device_eval_batch_size = 1,
  gradient_accumulation_steps = 16,
  eval_strategy = "no",
  predict_with_generate=True,
  generation_max_length = 128,
  save_steps = 500,
  logging_steps = 10,
  push_to_hub = False,
  max_steps=850
 )

In [17]:
trainer = Seq2SeqTrainer(
  model = model,
  args = training_args,
  data_collator = data_collator,
  train_dataset = tokenized_dataset["train"],
  tokenizer = tokenizer
 )

trainer.train()

  1%|          | 10/850 [01:34<2:11:33,  9.40s/it]

{'loss': 66.1439, 'grad_norm': 915.1886596679688, 'learning_rate': 5.555555555555556e-06, 'epoch': 0.03}


  2%|▏         | 20/850 [03:10<2:14:04,  9.69s/it]

{'loss': 65.5645, 'grad_norm': 589.9074096679688, 'learning_rate': 1.1111111111111112e-05, 'epoch': 0.05}


  4%|▎         | 30/850 [04:46<2:11:41,  9.64s/it]

...

100%|██████████| 850/850 [3:54:44<00:00, 14.52s/it]

{'loss': 4.209, 'grad_norm': 140.33139038085938, 'learning_rate': 0.0, 'epoch': 2.23}


100%|██████████| 850/850 [3:54:53<00:00, 16.58s/it]

{'train_runtime': 14093.0813, 'train_samples_per_second': 1.93, 'train_steps_per_second': 0.06, 'train_loss': 15.582739186006433, 'epoch': 2.23}


TrainOutput(global_step=850, training_loss=15.582739186006433, metrics={'train_runtime': 14093.0813, 'train_samples_per_second': 1.93, 'train_steps_per_second': 0.06, 'total_flos': 1.438093409255424e+16, 'train_loss': 15.582739186006433, 'epoch': 2.2262236045179242})

In [18]:
model.save_pretrained("D:/new_finetuned_model")
tokenizer.save_pretrained("D:/new_finetuned_model")

('D:/new_finetuned_model\\tokenizer_config.json',
 'D:/new_finetuned_model\\special_tokens_map.json',
 'D:/new_finetuned_model\\spiece.model',
 'D:/new_finetuned_model\\added_tokens.json')